In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from tqdm import tqdm

# 🔹 Meteor verisini yükle
meteor_df = pd.read_csv("../data/meteorite-landings.csv")

# GeoLocation varsa, lat/lon üret
meteor_df_geo = meteor_df.dropna(subset=['GeoLocation']).copy()
meteor_df_geo[['lat_from_geo', 'lon_from_geo']] = (
    meteor_df_geo['GeoLocation']
    .str.replace('[()]', '', regex=True)
    .str.split(',', expand=True)
    .astype(float)
)

# GeoDataFrame'e dönüştür
geometry = [Point(xy) for xy in zip(meteor_df_geo["lon_from_geo"], meteor_df_geo["lat_from_geo"])]
meteor_gdf = gpd.GeoDataFrame(meteor_df_geo, geometry=geometry, crs="EPSG:4326")

# 🔹 Yerleşim verisini shapefile'dan yükle
cities_gdf = gpd.read_file("../data/ne_10m_populated_places/ne_10m_populated_places.shp")

# EPSG:3857'e dönüştür (mesafe için gerekli)
cities_proj = cities_gdf.to_crs(epsg=3857)
meteor_proj = meteor_gdf.to_crs(epsg=3857)

# 🔹 Şehirler için en yakın meteor mesafesini hesapla
tqdm.pandas()
def nearest_meteor_distance(city_point, meteor_points):
    return meteor_points.geometry.distance(city_point).min()

cities_proj['min_meteor_distance_m'] = cities_proj.geometry.progress_apply(
    lambda x: nearest_meteor_distance(x, meteor_proj)
)


100%|█████████████████████████████████████████████████████████████████████████████| 7342/7342 [00:09<00:00, 799.39it/s]


In [2]:
from sklearn.cluster import KMeans
import numpy as np

# EPSG:4326 üzerinden KMeans için koordinatlar
coords = np.array(list(zip(meteor_gdf.geometry.x, meteor_gdf.geometry.y)))

kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
meteor_gdf['cluster'] = kmeans.fit_predict(coords)


In [3]:
# Gerekli klasör yoksa oluştur
import os
os.makedirs("../data/processed", exist_ok=True)

# Kaydet
cities_proj.to_file("../data/processed/cities_proj.geojson", driver="GeoJSON")
meteor_gdf.to_file("../data/processed/meteor_gdf.geojson", driver="GeoJSON")

print("Veriler başarıyla kaydedildi ve 'processed' klasörüne aktarıldı.")


Veriler başarıyla kaydedildi ve 'processed' klasörüne aktarıldı.


In [4]:
import pandas as pd
import geopandas as gpd
from sklearn.model_selection import train_test_split

# 1️⃣ Daha önce işlenmiş ve kaydedilmiş dosyaları yükle
cities_proj = gpd.read_file("../data/processed/cities_proj.geojson")
meteor_gdf = gpd.read_file("../data/processed/meteor_gdf.geojson")

# 2️⃣ Koordinat sistemini EPSG:4326'ya döndür (kıta tespiti için)
cities_deg = cities_proj.to_crs(epsg=4326).copy()

# 3️⃣ Kıta (continent) etiketlerini koordinatlara göre ata
def assign_continent(row):
    x, y = row.geometry.x, row.geometry.y
    if -30 <= x <= 50 and 35 <= y <= 70:
        return 'Europe'
    elif 50 <= x <= 150 and 0 <= y <= 60:
        return 'Asia'
    elif -170 <= x <= -30 and 10 <= y <= 80:
        return 'North America'
    elif -20 <= x <= 50 and -35 <= y <= 35:
        return 'Africa'
    elif -80 <= x <= -30 and -60 <= y <= -10:
        return 'South America'
    elif 110 <= x <= 180 and -50 <= y <= -10:
        return 'Australia'
    else:
        return 'Other'

cities_deg['continent'] = cities_deg.apply(assign_continent, axis=1)

# 4️⃣ Zaman bilgisi varsa, yıl ve on yıllık dönem (decade) ekle
if 'year' in cities_deg.columns:
    cities_deg['year'] = pd.to_numeric(cities_deg['year'], errors='coerce')
    cities_deg['decade'] = (cities_deg['year'] // 10 * 10).astype('Int64')  # eksikse boş kalır

# 5️⃣ Eksik mesafe verisini temizle
clean_data = cities_deg.dropna(subset=['min_meteor_distance_m'])

# 6️⃣ Kategorik: continent → dummies (model için uygun hale getirme)
clean_data = pd.get_dummies(clean_data, columns=['continent'], drop_first=True)

# 7️⃣ Hedef değişken: 500km altı şehirler yakın = 1, uzak = 0
clean_data['near_meteor'] = (clean_data['min_meteor_distance_m'] < 500_000).astype(int)

# 8️⃣ Girdi ve hedef değişkenlerin ayrılması
feature_cols = ['min_meteor_distance_m'] + [col for col in clean_data.columns if col.startswith('continent_')]
X = clean_data[feature_cols]
y = clean_data['near_meteor']

# 9️⃣ Eğitim/test seti oluştur
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 🔟 Özet
print("✅ Veri hazırlığı tamamlandı.")
print(f"🎯 Eğitim seti: {X_train.shape[0]} kayıt")
print(f"🧪 Test seti: {X_test.shape[0]} kayıt")
print(f"📊 Özellikler: {X_train.columns.tolist()}")


✅ Veri hazırlığı tamamlandı.
🎯 Eğitim seti: 5873 kayıt
🧪 Test seti: 1469 kayıt
📊 Özellikler: ['min_meteor_distance_m', 'continent_Asia', 'continent_Australia', 'continent_Europe', 'continent_North America', 'continent_Other', 'continent_South America']
